# GraphFlood example

This example explains how to use the "full" method to calculate flow depth and discharge on a DEM from precipitation rates

In [ ]:
import scabbard as scb
import dagger as dag
import numpy as np
import matplotlib.pyplot as plt
import math as m
import matplotlib
import random
from scipy.ndimage import gaussian_filter
from scabbard import ModelHelper, PlotHelper
%matplotlib widget

# Name of the DEM to load
fnamedem = "dem.tif"

# Graphflood object
mod = ModelHelper()

# precipitation rates in m/s
# P = 1e-4
# P = 2.7778e-6 #  => 10 mm/h
# P = 2.7778e-6/2 #  => 5 mm/h
# P = 5.5556e-6 #  => 20 mm/h
# P = 1.3889e-5 # => 30
# P = 1.1111e-5 # => 40 mm/h
# P = 1.3889e-5 # => 50 mm/h
# P = 2.7778e-5 # => 100 mm/h

P = 8.333e-6 # Precipitation rate in m/s (this is super high just for fun)
P = 50 * 1e-3/3600
mod.init_dem_model(fnamedem, sea_level = 0., P = P) 

# Use CFL conditions to adapt timestep
mod.courant = True
mod.min_courant_dt = 1e-6 # Minimum timestep if CFL dt
mod.courant_number = 0.04 # Courant number modulates dt: higher courant = higher dt. Depends on many things (dx, P, ...)

# if courant is False, this will be used as time step
mod.dt = 5e-3

# Stationary = run to equillibrium
# Stationary = False propagate a trasient flood wave (the model has not been developped for that though)
mod.stationary = True 

# manning friction coeff, 0.033 is an OK value for open flow in channels:
mod.mannings = 0.033

# Single flow solver or Multiple flow solver?
# SFD: faster, less accurate
# MFD: slower, more accurate and prettier
mod.SFD = False

# Plot a dynamic figure (also works outside of jupyter)
ph = PlotHelper(mod, jupyter = True)
ph.init_hw_plot(use_extent = True, vmax = 0.6)

In [ ]:
# Update the above figure every X timesteps
update_fig = 20
# Total number of run for this time step
Ndt = 10000
for i in range(Ndt):
    # Run one dt
    mod.run()
    if(i % update_fig > 0):
        continue
    print(i, end = '\r')
    # Update figure
    ph.update()

# Custom plot

The following example shows how to get the field of flow depth from the model and visualise it with matplotlib

In [ ]:
#Getting the actual 2D array of flow depth
flow_depth = mod.hw

flow_depth[flow_depth < 0.05] = np.nan

fig, ax = plt.subplots(figsize = (8,8))
imhs = ax.imshow(mod.grid.hillshade, extent = mod.grid.extent(), cmap = 'gray') # plots an hiilshade of the relief. you can also access elevation with mod.grid.Z2D
imhw = ax.imshow(flow_depth, cmap = "Blues", extent = mod.grid.extent(), vmin = 0, vmax = 1)
plt.colorbar(imhw, label = r"$h_W$ (m)")
ax.set_xlabel("Easting (m)")
ax.set_ylabel("Northing (m)")
plt.tight_layout()